Updating stations in OSCAR/Surface
---
> **Example #3:** We will update the schedules of German stations. We want to update stations that only have a _30 min reporting interval_ . Their reporting interval  should be changed to 1h and the international exchange flag set to _true_. In case there is no information on start and endtime of the schedule, set it to correspond to a 7/24 schedule.

***Note***: we limit ourselves to observations tracked by WDMQS for the purpose of this exercise. We thus look for **ongoing** observations of _pressure, temperature, wind_ and _humidity_ at stations.




In [6]:
import json
import sys
import logging
logging.basicConfig(level=logging.INFO)

from oscar_lib import OscarClient, extractSchedules, Station

wdqms_variables = [216,210,224,251,309,12005,12006] # pressure,precipitation,temperature,humidity,wind deprecated, wind speed, wind direction

In [11]:
client = OscarClient(oscarurl = OscarClient.OSCAR_DEPL, token="248eadae-8d38-4410-85c9-ccfd0ba897fa")
stations=client.oscarSearch({'territoryName':'DEU', 'facilityType':'landFixed'})

# collect primary wigos ids from search result. 
# Need to loop through search result and potential multiple wigos ids to find the primary one
german_station_wigosids = [ wid["wigosStationIdentifier"] for station in stations["data"] for wid in station["wigosStationIdentifiers"] if wid["primary"] ]

german_station_wigosids

INFO:oscar_lib.oscar_client:searching for {'territoryName': 'DEU', 'facilityType': 'landFixed'} at https://oscardepl.wmo.int/surface//rest/api/search/station


['0-20000-0-89047',
 '0-20000-0-89003',
 '0-20000-0-89002',
 '0-20008-0-NMY',
 '0-20000-0-89011',
 '0-20000-0-10505',
 '0-20000-0-10309',
 '0-20000-0-10442',
 '0-20000-0-10954',
 '0-20000-0-10520',
 '0-20000-0-10291',
 '0-20000-0-10091',
 '0-20000-0-10460',
 '0-20000-0-10852',
 '0-20008-0-DAN',
 '0-20008-0-AKN',
 '0-20008-0-DAU',
 '0-20000-0-10542',
 '0-20000-0-10658',
 '0-20000-0-10430',
 '0-20000-0-10526',
 '0-20000-0-10325',
 '0-20000-0-10675',
 '0-20000-0-10180',
 '0-20000-0-10376',
 '0-20000-0-10312',
 '0-20000-0-10238',
 '0-20000-0-10381',
 '0-20000-0-10385',
 '0-20000-0-10382',
 '0-20000-0-10384',
 '0-20000-0-10704',
 '0-20000-0-10249',
 '0-20000-0-10161',
 '0-20000-0-10519',
 '0-20000-0-10132',
 '0-276-0-10103',
 '0-20000-0-10452',
 '0-20000-0-10348',
 '0-20000-0-10224',
 '0-20000-0-10129',
 '0-20000-0-10139',
 '0-20000-0-10453',
 '0-20000-0-10613',
 '0-20000-0-10335',
 '0-20008-0-BSU',
 '0-20000-0-10678',
 '0-20000-0-10389',
 '0-20008-0-BRL',
 '0-20008-0-BEM',
 '0-20008-0-BRT'

In [12]:
limit=10
station_map = {}
for wigos_id in german_station_wigosids[0:min(limit,len(german_station_wigosids))]:
    try:
        myxml = client.load_station(wigos_id=wigos_id)
        s = Station(myxml)
        
        if "20008" in wigos_id: # ignore GAWSIS stations
            continue
        
        # check if the station has at least one ongoing 30 min schedule (for WDQMS relevant variables)
        schedules = s.current_schedules(variables=wdqms_variables)

        # loop through nested datastructure and check if interval is 30min
        intervals = [ dg["schedule"]["interval"] == "PT30M" for o in schedules.values() for d in o["deployments"] for dg in d["datagenerations"] ]

        if any(intervals): # we consider the station as long as there is one 30min schedule
            station_map[wigos_id]=s
    
    except Exception as e:
        print("error loading station {} {}".format(wigos_id,str(e)))


In [13]:
station_map

{'0-20000-0-10309': <oscar_lib.station.Station at 0xad57438>,
 '0-20000-0-10442': <oscar_lib.station.Station at 0x7a305c0>,
 '0-20000-0-10520': <oscar_lib.station.Station at 0xad63e48>}

### update strategy: 
 1. set 1h international 7/24 schedule for "ongoing" schedules. 
 2. other schedules will be merged with the same 1h international 7/24 schedule, preserving the elements present in the original schedule



In [14]:
defaultSchedule = { 
    "from": "2016-04-28T00:00:00Z", "to" : None,     
    "startMonth": 1,"endMonth": 12,
    "startWeekday": 1, "endWeekday": 7,
    "startHour": 0, "startMinute": 0,          
    "endHour": 23,    "endMinute": 59,
    "interval": "PT1H",     "international": True   
}


In [ ]:
for wigos_id,station in station_map.items():
    print("updating station",wigos_id)
    
    current_schedules = station.current_schedules() # get "ongoing" schedules
    for current_schedule in current_schedules.values():
        for d in current_schedule["deployments"]:
            for dg in d["datagenerations"]:
                gid=dg["gid"]
                schedule=dg["schedule"]
                
                station.fix_and_update_datageneration(gid,defaultSchedule) 

    try:
        station.validate()
        print("validates ok")
    except: # if there are remaining validation issues
        print("fixing remaining schedules")
        station.fix_deployments(mode="update",defaultSchedule=defaultSchedule) #merge remaining eronous schedules

    station.validate()
    
    
    station.save(client)
    print("saved",wigos_id)
    